<a href="https://colab.research.google.com/github/phuonghoathu/nothing1988nevergive/blob/main/L3_AdvancedRAG_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
openai_api_key = ""

# Cài đặt thư viện cần thiết

In [ ]:
!pip install --quiet -U langchain chromadb langchain-openai pypdf gradio

# Xử lí văn bản PDF

In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
loader = PyPDFLoader("/content/VinaLLaMA_final.pdf")
splits = loader.load_and_split(text_splitter)

# Đưa văn bản thành Embeddings và Lưu trữ vào vectorstore

In [ ]:
emb = OpenAIEmbeddings(openai_api_key=openai_api_key)
vectorstore = Chroma.from_documents(documents=splits, embedding=emb)

In [ ]:
splits[1]

Document(page_content='VinaLLaMA-7B-chat, trained on 1-million high quality synthetic samples, achieves SOTA results\non key benchmarks, including VLSP, VMLU, and Vicuna Benchmark Vietnamese, marking a\nsignificant advancement in the Vietnamese AI landscape and offering a versatile resource for various\napplications.\n1 Introduction\nThe surge in Large Language Models (LLMs) such as ChatGPT and GPT-4 has significantly advanced\nthe field of artificial intelligence (AI), particularly in language processing. In 2023, Vietnam’s AI', metadata={'source': '/content/VinaLLaMA_final.pdf', 'page': 0})

# Xây Dựng Gradio

In [ ]:
import gradio as gr
from openai import OpenAI

llm = OpenAI(api_key=openai_api_key)

RAG_PROMPT = """Nhiệm vụ của bạn là trả lời câu hỏi của người dùng dựa trên dữ liệu được cho.
Nếu dữ liệu được cho không liên quan đến câu hỏi, vui lòng trả lời "Tôi không biết"
---
Dữ liệu: {context}
---
Câu hỏi: {question}
---
Trả lời:"""

RAG_PROMPT_HISTORY = """Nhiệm vụ của bạn là trả lời câu hỏi của người dùng dựa trên dữ liệu được cho.
Nếu dữ liệu được cho không liên quan đến câu hỏi, vui lòng trả lời "Tôi không biết"
---
Dữ liệu: {context}
---
Chat History:
{history}
---
Câu hỏi: {question}
"""

def build_history(history):
    history_str = ""
    for human, assistant in history:
        history_str += f"Human: {human}\nAssistant: {assistant}\n"
    return history_str

def predict(message, history):
    history_openai_format = []
    if len(history) == 0:
      for human, assistant in history:
          history_openai_format.append({"role": "user", "content": human })
          history_openai_format.append({"role": "assistant", "content": assistant})
      docs = vectorstore.similarity_search(message)
      context = docs[0].page_content
      history_openai_format.append({"role": "user", "content": RAG_PROMPT.format(context=context, question=message)})
    else:
      history_str = build_history(history)
      docs = vectorstore.similarity_search(history_str)
      context = docs[0].page_content
      history_openai_format.append({"role": "user", "content": RAG_PROMPT_HISTORY.format(context=context, history=history_str, question=message) })


    response = llm.chat.completions.create(model='gpt-3.5-turbo',
      messages= history_openai_format,
      temperature=1.0,
      stream=True)

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message



gr.ChatInterface(predict).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2963efa5dbd623512f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ReadTimeout: The read operation timed out